In [16]:
# set up non-QM hardware
import Labber
# connect to server 
client = Labber.connectToServer('localhost') # get list of instruments 
#instruments = client.getListOfInstrumentsString() 
#for instr in instruments: 
#    print(instr) # close connection 

# reset all QDevil channels to 0 V
QDevil = client.connectToInstrument('QDevil QDAC', dict(interface='Serial', address='3'))
for n in range(24):
    if n+1 < 10:
        QDevil.setValue("CH0" + str(n+1) + " Voltage", 0.0)
    else:
        QDevil.setValue("CH" + str(n+1) + " Voltage", 0.0)
# setting CH06 to sweet spot
DC_ss = 6.4352E-3
QDevil.setValue("CH06 Voltage", DC_ss)

# digital attenuators
Vaunix1 = client.connectToInstrument('Painter Vaunix Lab Brick Digital Attenuator', dict(interface='USB', address='25606'))
Vaunix2 = client.connectToInstrument('Painter Vaunix Lab Brick Digital Attenuator', dict(interface='USB', address='25607'))
ROI = 42
ROO = 10
Vaunix1.setValue("Attenuation", ROI)
Vaunix2.setValue("Attenuation", ROO)

# TWPA pump
SG = client.connectToInstrument('Rohde&Schwarz RF Source', dict(interface='TCPIP', address='192.168.88.2'))
SG.setValue('Frequency', 6730.6E6)
SG.setValue('Power', -11.3)

client.close()

In [17]:
from qm.qua import *
from qm.QuantumMachinesManager import QuantumMachinesManager
from configuration import *
import matplotlib.pyplot as plt
import numpy as np
from qm import SimulationConfig
from qualang_tools.loops import from_array
%matplotlib qt

# Start to save data
import datetime 
import os
import pandas as pd 

print(datetime.datetime.now(),1)

now = datetime.datetime.now()
year = now.strftime("%Y")
month = now.strftime("%m")
day = now.strftime("%d")
hour = now.strftime("%H")
minute = now.strftime("%M")
second  = now.strftime("%S")

tPath = os.path.join(r'Z:\LabberData_DF5\QM_Data_DF5',year,month,'Data_'+month+day)
if not os.path.exists(tPath):
   os.makedirs(tPath)
exp_name = 'T1'
qubit_name = 'Q2'

f_str =exp_name + '_'+ qubit_name +'_'+ hour+ '_' + minute +'_' + second
f_name= f_str+'.csv'
j_name = f_str + '_state.json'
p_name= f_str+'.png'

# End to save data

2023-08-17 17:34:53.421982 1


In [18]:
machine = QuAM("quam_state.json")
config = build_config(machine)

###################
# The QUA program #
###################

tau_min = 100 # clock cycle
tau_max = 5_000 # clock cycle
d_tau = 100 # clock cycle
taus = np.arange(tau_min, tau_max + 0.1, d_tau)  # + 0.1 to add t_max to taus
qubit_index = 5

n_avg = 20000
cooldown_time = 50_000 * u.ns

with program() as T1:
    n = declare(int)
    n_st = declare_stream()
    I = declare(fixed)
    I_st = declare_stream()
    Q = declare(fixed)
    Q_st = declare_stream()
    tau = declare(int)

    with for_(n, 0, n < n_avg, n + 1):
        with for_(*from_array(tau, taus)):
            play("x180", machine.qubits[qubit_index].name)
            wait(tau, machine.qubits[qubit_index].name)
            align(machine.qubits[qubit_index].name, machine.resonators[qubit_index].name)
            measure(
                "readout",
                machine.resonators[qubit_index].name,
                None,
                dual_demod.full("rotated_cos", "out1", "rotated_sin", "out2", I),
                dual_demod.full("rotated_minus_sin", "out1", "rotated_cos", "out2", Q),
            )
            save(I, I_st)
            save(Q, Q_st)
            wait(cooldown_time * u.ns, machine.resonators[qubit_index].name)
        save(n, n_st)

    with stream_processing():
        I_st.buffer(len(taus)).average().save("I")
        Q_st.buffer(len(taus)).average().save("Q")
        n_st.save("iteration")

#####################################
#  Open Communication with the QOP  #
#####################################
qmm = QuantumMachinesManager(machine.network.qop_ip, cluster_name=machine.network.cluster_name, octave=octave_config)

#######################
# Simulate or execute #
#######################
simulate = False

if simulate:
    simulation_config = SimulationConfig(duration=20_000)  # in clock cycles
    job = qmm.simulate(config, T1, simulation_config)
    job.get_simulated_samples().con1.plot()

else:
    print(datetime.datetime.now(),2)
    qm = qmm.open_qm(config)
    job = qm.execute(T1)
    
    # Get results from QUA program
    results = fetching_tool(job, data_list=["I", "Q", "iteration"], mode="live")
    # Live plotting
#     fig = plt.figure()
#     plt.rcParams['figure.figsize'] = [8, 6]
#     interrupt_on_close(fig, job)  # Interrupts the job when closing the figure
    while results.is_processing():
        # Fetch results
        I, Q, iteration = results.fetch_all()
        # Progress bar
        progress_counter(iteration, n_avg, start_time=results.get_start_time())
        # Plot results
        if False:
            plt.cla()
            plt.plot(4 * taus, I, ".", label="I")
            plt.plot(4 * taus, Q, ".", label="Q")
            plt.xlabel("Decay time [ns]")
            plt.ylabel("I & Q amplitude [a.u.]")
            plt.title("T1 measurement")
            plt.legend()
            plt.pause(0.1)
print(datetime.datetime.now(),3)

2023-08-17 17:35:10,742 - qm - ERROR    - Encountered connection error from QOP
Traceback (most recent call last):
  File "C:\Users\painter\AppData\Roaming\Python\Python38\site-packages\qm\api\base_api.py", line 27, in wrapped
    return func(*args, **kwargs)
  File "C:\Users\painter\AppData\Roaming\Python\Python38\site-packages\qm\api\frontend_api.py", line 79, in get_version
    response = run_async(self._stub.get_version(Empty(), timeout=self._timeout))
  File "C:\Users\painter\AppData\Roaming\Python\Python38\site-packages\qm\utils\async_utils.py", line 37, in run_async
    return create_future(coroutine).result()  # type: ignore[no-any-return]
  File "C:\Users\painter\.conda\envs\labber-qm-env\lib\concurrent\futures\_base.py", line 444, in result
    return self.__get_result()
  File "C:\Users\painter\.conda\envs\labber-qm-env\lib\concurrent\futures\_base.py", line 389, in __get_result
    raise self._exception
  File "C:\Users\painter\AppData\Roaming\Python\Python38\site-packages\

In [20]:
# Fetch results
I, Q, iteration = results.fetch_all()
# print('iteration', iteration)
# Convert I & Q to Volts
I = u.demod2volts(I, machine.resonators[qubit_index].readout_pulse_length)
Q = u.demod2volts(Q, machine.resonators[qubit_index].readout_pulse_length)
sigs = np.sqrt(I**2 + Q**2)

# fit the data
from scipy.optimize import curve_fit

def exp_decay(t, A, T1, c):
    return A*np.exp(-t/T1)+c
    
try:
    param, _ = curve_fit(exp_decay,taus,sigs,p0=[max(sigs)-min(sigs),4E3,min(sigs)] )
    q_T1 = round(param[1])
    print('Qubit T1 [us]:', q_T1 * 4.0*u.ns/u.us)
except:
    pass

# 1D spectroscopy plot
fig = plt.figure(figsize=[14, 6])
plt.plot(taus * 4.0*u.ns /u.us, sigs, "b.")
plt.plot(taus * 4.0*u.ns / u.us, exp_decay(taus, param[0],param[1],param[2]), 'r')
plt.show()
plt.title("Qubit T1")

plt.xlabel("tau [us]")
plt.ylabel(r"$\sqrt{I^2 + Q^2}$ [V]")

Qubit T1 [us]: 5.04


Text(0, 0.5, '$\\sqrt{I^2 + Q^2}$ [V]')

In [21]:
plt.savefig(os.path.join(tPath, p_name))
data = np.asarray([taus, I,Q])
df = pd.DataFrame(data)
df.index = ['tau (clock cycle)', 'I', 'Q']
df.to_csv(os.path.join(tPath, f_name),header=False, index=False)
open(os.path.join(tPath, j_name), "w").write(open("quam_state.json").read())

## convert everything to lists so they are easy to parse in Matlab if desired
if isinstance(taus,np.ndarray) == True:
   taus=taus.tolist()
else:  
    pass
if isinstance(I,np.ndarray) == True:
   I=I.tolist()
else:  
    pass
if isinstance(Q,np.ndarray) == True:
   Q=Q.tolist()
else:  
    pass

data = [('Variable', 'Value'), ('Delay', taus), ('I', I), ('Q', Q)]
df = pd.DataFrame(data,index=None, columns=None)
df.to_csv(os.path.join(tPath, f_name),header=False, index=False)
open(os.path.join(tPath, j_name), "w").write(open("quam_state.json").read())

9051

In [22]:
f_name

'T1_Q2_17_34_53.csv'